In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from cv2 import imread

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F, CrossEntropyLoss
from torch.optim import Adam, SGD
from torch.utils.data import TensorDataset, DataLoader, Dataset
from albumentations import HorizontalFlip, VerticalFlip, Resize, Compose
from albumentations.augmentations.transforms import Normalize

import torchvision


torch.manual_seed(0)
np.random.seed(0)

In [3]:
import wandb

wandb.init(project="NeuroWood2022", entity="f3ss1")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: f3ss1 (use `wandb login --relogin` to force relogin)


In [4]:
torch.cuda.empty_cache()

class SETUP:
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 4
    SIZE = 1024

def transform(x):
    if (x == 0):
        return x
    elif (x == 1):
        return x
    else:
        return 3

In [5]:
data = pd.read_csv('train.csv')
data = data.sort_values('id').reset_index().drop('index', axis=1)
data['class'] = data['class'].astype(int)
data.head()

,id,class
0,IMG_0581,1
1,IMG_0583,1
2,IMG_0584,1
3,IMG_0585,1
4,IMG_0589,1


In [6]:
output_dataframe = pd.read_csv('sample_submission.csv')
output_dataframe['id'] = output_dataframe['id'].astype(str)
output_dataframe.head()

,id,class
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN


In [7]:
class WoodDataset(Dataset):
    def __init__(self, create_train=True) -> None:
        
        self.transform = Compose([Resize(SETUP.SIZE, SETUP.SIZE), Normalize()])
        self.create_train = create_train
        
        
    
    def __getitem__(self, index) -> dict:
        
        if self.create_train:
            id = data.iloc[index]['id']
            image = imread(f'Data/Train/{id}.png').astype(np.float32)
            processed_image = self.transform(image=image)
            return np.moveaxis(processed_image['image'], 2, 0), torch.tensor(data.iloc[index]['class'], dtype=torch.int64)
        else:
            id = output_dataframe.iloc[index]['id']
            image = imread(f'Data/Test/{id}.png').astype(np.float32)
            processed_image = self.transform(image=image)
            return np.moveaxis(processed_image['image'], 2, 0)

            

    def __len__(self) -> None:
        if self.create_train:
            return data.shape[0]
        else:
            return output_dataframe.shape[0]

In [8]:
train_dataset = WoodDataset()
test_dataset = WoodDataset(create_train=False)

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=SETUP.BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=SETUP.BATCH_SIZE)

In [10]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device=SETUP.DEVICE):
    rolling_loss = 0
    for images, labels in tqdm(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        y_pred = model.forward(images)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        rolling_loss += loss.detach().cpu().numpy()
    return rolling_loss / len(train_dataloader)

def train(model, train_dataloader, criterion, optimizer, device=SETUP.DEVICE, n_epochs=10):
    model.to(device)
    model.train()
    for _ in range(n_epochs):
        loss = train_one_epoch(model, train_dataloader, criterion, optimizer, device)
        wandb.log({"loss": loss})

def predict(model, test_dataloader, device=SETUP.DEVICE):
    model = model.to(device)
    predicted_labels = torch.tensor([]).to(device)
    for images in tqdm(test_dataloader):
        images = images.to(device)
        y_pred = model.forward(images)
        predicted_labels = torch.cat((predicted_labels, y_pred.argmax(1).detach()), 0)
    return predicted_labels.cpu()

In [11]:
class ModelFinal(nn.Module):
    def __init__(self):
        super(ModelFinal, self).__init__()
        self.model = nn.Sequential(
            torchvision.models.resnet18(pretrained=True),
            nn.ReLU(),
            nn.Linear(1000, 3),
            nn.Softmax(dim=0)
        )
    def forward(self, x):
        return self.model.forward(x)

In [12]:
model = ModelFinal()

parameters = {
    'optimizer' : SGD(model.parameters(), lr=0.01),
    'criterion' : CrossEntropyLoss(),
    'device' : SETUP.DEVICE,
    'n_epochs' : 10
}

wandb.config = {
  "learning_rate": "default",
  "epochs": parameters['n_epochs'],
  "batch_size": SETUP.BATCH_SIZE,
  "Network": "Resnet18 + Softmax"
}

In [13]:
train(model, train_dataloader, **parameters)

100%|██████████| 145/145 [02:29<00:00,  1.03s/it]


In [14]:
predicts = predict(model, test_dataloader)

100%|██████████| 63/63 [00:58<00:00,  1.07it/s]


In [15]:
predicts

tensor([1., 2., 0., 2., 0., 1., 2., 0., 2., 1., 0., 1., 0., 2., 1., 1., 1., 2.,
        0., 0., 1., 2., 2., 2., 1., 2., 0., 2., 2., 1., 1., 0., 1., 0., 2., 1.,
        2., 1., 1., 0., 1., 0., 2., 0., 1., 2., 0., 2., 1., 0., 1., 2., 1., 2.,
        2., 0., 2., 1., 0., 1., 2., 2., 1., 2., 0., 2., 1., 2., 2., 1., 2., 0.,
        2., 0., 0., 1., 2., 0., 1., 2., 2., 0., 1., 1., 2., 2., 1., 0., 0., 2.,
        1., 1., 0., 2., 2., 1., 1., 0., 2., 2., 0., 2., 1., 1., 1., 1., 2., 1.,
        0., 2., 2., 1., 0., 0., 2., 1., 2., 0., 1., 2., 2., 0., 1., 1., 0., 2.,
        1., 1., 1., 1., 2., 0., 2., 2., 0., 1., 1., 0., 0., 2., 0., 2., 1., 0.,
        2., 2., 0., 1., 0., 1., 1., 1., 2., 1., 1., 1., 2., 2., 1., 0., 2., 0.,
        2., 1., 0., 0., 1., 1., 0., 2., 2., 1., 1., 2., 1., 0., 2., 1., 2., 1.,
        0., 2., 1., 1., 0., 1., 2., 0., 2., 1., 0., 1., 2., 0., 2., 1., 2., 1.,
        2., 2., 1., 0., 0., 2., 1., 0., 2., 1., 0., 1., 0., 2., 1., 0., 2., 2.,
        1., 2., 1., 0., 2., 1., 0., 1., 

In [16]:
output_dataframe['class'] = predicts
output_dataframe['class'] = output_dataframe['class'].astype(int)
output_dataframe['class'] = output_dataframe['class'].apply(transform)
output_dataframe.to_csv('prediction.csv', index=False)